In [48]:
###Importation des packages

import numpy as np
import time

In [2]:
def generation(n, alpha=2, beta=0, gamma=1, delta=0):
    y = [0]*n
    tan = np.tan(np.pi*alpha/2)
    S = (1 + beta**2 * tan**2) ** (1 / (2 * alpha))
    B = (1 / alpha) * np.arctan(beta * tan)
    #print("S:", S, "B:", B, "tan(pi*alpha/2)", tan)

    for i in range(n):
        w = -np.log(1 - np.random.rand())
        u = np.random.uniform(-np.pi/2, np.pi/2)
        
        if alpha == 1:
            #print("before transf:", y[i])
            y[i] = 2 / np.pi * ((np.pi/2 + beta * u) * np.tan(u) - beta * np.log((np.pi/2 * w * np.cos(u)) / (np.pi/2 + beta * u)))
            #print("after transf:", y[i])
        else:
            #print("before transf:", y[i])
            y[i] = S * np.sin(alpha) * (u + B) / (np.cos(u)**(1 / alpha)) * (np.cos(u - alpha * (u + B)) / w) ** ((1 - alpha) / alpha)
            #print("after transf:", y[i])

        #print("before scaling:", y[i])
        y[i] = gamma * y[i] + delta
        #print("after scaling:", y[i])
    return y

$\theta = (\alpha, \beta, \gamma, \delta)$ 

N = 1000

$x^1,...,x^p$ ~ $\pi(x|\theta)$

$\pi_{LF}(\theta|y)$ = $\pi(\theta)\mathbb{E}_{\pi(x|\theta)}[K_{\epsilon} (y-x)]$ $\leftarrow$ we do monte carlo on this

$K_{\epsilon} (y-x)$ def by $S(y)$ ~ $N(S(x), \epsilon^2 \hat{\Sigma})$ $\leftarrow$ that would be our gaussian kernel

$\hat{\Sigma}$ is an estimate of $Cov(S(x)\hat{\theta})$

$\Lambda$ = diag(0.25, 0.25, 1, 1)

Mutation kernel : $M_t(\theta_t) = \sum_{i=1}^{N} W_{t-1}^{(i)}(\theta_{t-1}^{(i)}) \phi(\theta_t ; \theta_{t-1}^{(i)})$

Gaussian kernel : $L(x,t) = \sum_{n = -M}^{M}f(x-n)G(n,t)$

with G(n,t) = $\frac{1}{\sqrt{2\pi t}}e^{-\frac{n^2}{2t}}$


As t increases, $\epsilon_t$ decreases

$\hat{c_t}$ is the 90th quantile of the weights

In [3]:
{}

{}

# Seed : 256652

In [4]:
np.random.seed(256652)

#creation des prior des parametres cf section 3.1

prior_alpha = np.random.uniform(1.1, 2., size=1000)
prior_beta = np.random.uniform(-1., 1, size=1000)
prior_gamma = np.random.uniform(0., 300., size=1000)
prior_delta = np.random.uniform(-300., 300., size=1000)

#scale parameters

#epsilon_t

mille = np.linspace(start=100, stop=1000, endpoint=True, num=10)[::-1]
cent = np.linspace(start=10, stop=100, endpoint=False, num=90)[::-1]
dix = np.linspace(start=5, stop=10, endpoint=False, num=10)[::-1]
cinq = np.linspace(start=3, stop=5, endpoint=False, num=40)[::-1]
trois = np.linspace(start=0, stop=3, endpoint=False, num=300)[::-1]

scale_param = np.concatenate((mille, cent, dix, cinq, trois))


N = 1000
varcov_lambda = np.array([0.25,0.,0.,0.,0.,0.25, 0., 0.,0.,0.,1.,0.,0.,0.,0.,1]).reshape([4,4])

In [5]:
def mc_culloch_q(data, gamma=1):
    """fonction pour calculer les quantiles de mc_culloch utilises dans le papier

    Parameters
    --------------
    data : array
    les donnes pour lesquelles on veut calculer les quantiles

    gamma : float
    valeur 1 par defaut, d'apres le papier, on utilise le gamma utilise pour faire les simulations
    ----------------
    """
    data_sorted = np.sort(data)
    useful_quantiles = np.quantile(a=data_sorted, q=[0.95, 0.75, 0.5, 0.25, 0.05])
    q_95 = useful_quantiles[0]
    q_75 = useful_quantiles[1]
    q_50 = useful_quantiles[2]
    q_25 = useful_quantiles[3]
    q_05 = useful_quantiles[4]
    alpha_hat = (q_95-q_05)/(q_75-q_25)
    beta_hat = (q_95+q_05+2*q_50)/(q_95-q_05)
    gamma_hat = (q_75-q_25)/gamma
    #on a le gamma au denominateur parce que c'est comme ca qu on a genere nos donnees
    #d'apres le papier, ca va de prendre le gamma qu'on a utililse pour generer
    delta_hat = np.mean(data_simulation)
    S_1 = np.transpose(np.array((alpha_hat, beta_hat, gamma_hat, delta_hat)))
    return S_1

In [6]:
2.

2.0

In [7]:
4.75e+00

4.75

In [8]:
%%capture
#data generation and definition of moments

data_simulation = generation(n=1000, alpha=2, beta=0.5, gamma=3, delta=4)

In [9]:
%%capture
#sort data to be sure the quantiles are well computed bc you never know
data_simulation = np.sort(data_simulation)

In [10]:
useful_quantiles = np.quantile(a=data_simulation, q=[0.95, 0.75, 0.5, 0.25, 0.05])

In [11]:
useful_quantiles

array([ 23.92205984,   6.10073993,   3.84557563,   1.70704736,
       -12.25474987])

In [12]:
q_95 = useful_quantiles[0]
q_75 = useful_quantiles[1]
q_50 = useful_quantiles[2]
q_25 = useful_quantiles[3]
q_05 = useful_quantiles[4]

### Mc Culloch's quantiles

In [13]:
alpha_hat = (q_95-q_05)/(q_75-q_25)
beta_hat = (q_95+q_05+2*q_50)/(q_95-q_05)
gamma_hat = (q_75-q_25)/3
#on a le 3 au denominateur parce que c'est comme ca qu on a genere nos donnees
#d'apres le papier, ca va de prendre le gamma qu'on a utililse pour generer
delta_hat = np.mean(data_simulation)

In [14]:
S_1 = np.transpose(np.array((alpha_hat, beta_hat, gamma_hat, delta_hat)))

In [15]:
S_1

array([8.23380542, 0.53510692, 1.46456419, 0.50091662])

In [16]:
delta_hat

0.5009166248083248

Test runned with one million simulations, the estimators seemed to be biased.

In [17]:
def prior_lf_mc():
    "estimation monte carlo de la vraisemblance
    

SyntaxError: unterminated string literal (detected at line 2) (2212262960.py, line 2)

In [ ]:
def smc_prc_abc(N=1000, epsilon_t):
    """Algorithme ABC de l'appendix A de l'article.


    Parameters
    ---------------------------
    N : int
    Nombre d'echantillons crees pour les parametres

    epsilon_t : array
    Les marges d'acceptation pour l'algorithme
    """
    #les prior sont des uniformes cf partie 3.1
    prior_alpha = np.random.uniform(1.1, 2., size=N)
    prior_beta = np.random.uniform(-1., 1, size=N)
    prior_gamma = np.random.uniform(0., 300., size=N)
    prior_delta = np.random.uniform(-300., 300., size=N)
    prior_gen = np.vstack((prior_alpha,prior_beta,prior_delta, prior_gamma))
    
    #the weights
    

    

    

In [18]:
a = np.array((1,2,3,4,5))

In [19]:
a

array([1, 2, 3, 4, 5])

In [20]:
b = np.array((5,4,3,2,1))

In [21]:
a/b

array([0.2, 0.5, 1. , 2. , 5. ])

In [22]:
c = np.array((5,1,3,2,4))

In [23]:
test = np.vstack((a,b,c))

In [24]:
test

array([[1, 2, 3, 4, 5],
       [5, 4, 3, 2, 1],
       [5, 1, 3, 2, 4]])

In [31]:
np.linalg.norm(a - b)

6.324555320336759

In [25]:
test[:,0]

array([1, 5, 5])

In [44]:
def accept_reject_abc(N=1000, epsilon_t=0.5, alpha=2, beta=0, gamma=1, delta=0):
    """algorithme d'acceptation rejet 'basique', vu en cours

    Parameters
    ---------------------------------
    N : int
    nombre de simulations faites

    epsilon_t : array
    scale parameters, marge d'acceptation

    alpha, beta, gamma, delta :
    parametres pour les simulations
    -------------------------------
    """
    data = generation(n=N, alpha=alpha, beta=beta, gamma=gamma, delta=delta)
    #priors
    prior_alpha = np.random.uniform(1.1, 2., size=N)
    prior_beta = np.random.uniform(-1., 1, size=N)
    prior_gamma = np.random.uniform(0., 300., size=N)
    prior_delta = np.random.uniform(-300., 300., size=N)
    prior_gen = np.vstack((prior_alpha,prior_beta,prior_delta, prior_gamma))
    true_data_estim = mc_culloch_q(data, gamma=gamma)
    bon_param = np.array(())
    alpha_test = 0
    beta_test = 0
    gamma_test = 0
    delta_test = 0
    for i in range(N):
        proposed_data = generation(n=N, alpha=prior_gen[:,i][0],
                                   beta=prior_gen[:,i][1],
                                   delta=prior_gen[:,i][2],
                                   gamma=prior_gen[:,i][3])
        alpha_test = prior_gen[:,i][0]
        beta_test = prior_gen[:,i][1]
        gamma_test = prior_gen[:,i][2]
        delta_test = prior_gen[:,i][3]
        estimated = mc_culloch_q(proposed_data, gamma=prior_gen[:,i][3])
        if np.linalg.norm(estimated - true_data_estim) < epsilon_t:
            return alpha_test, beta_test, gamma_test, delta_test
        else :
            continue

In [29]:
    prior_alpha = np.random.uniform(1.1, 2., size=N)
    prior_beta = np.random.uniform(-1., 1, size=N)
    prior_gamma = np.random.uniform(0., 300., size=N)
    prior_delta = np.random.uniform(-300., 300., size=N)
    prior_gen = np.vstack((prior_alpha,prior_beta,prior_delta, prior_gamma))

In [45]:
test_1 = accept_reject_abc(N=1000, epsilon_t=0.3)

In [53]:
start = time.time()
test_1
end = time.time()
print(test_1, end - start)

(1.9796634508883522, 0.14873567823579426, -153.78056311673896, 203.5630079295496) 7.271766662597656e-05


In [36]:
#avec une si petite distance ca ne marchait pas

In [54]:
start = time.time()
liste_parametres = []
for i in range(len(scale_param)):
    pouet = accept_reject_abc(N=1000, epsilon_t=scale_param[-i])
    intermediaire = [scale_param[-i], pouet]
    liste_parametres.append(intermediaire)

end = time.time()

print(end - start)

268.54338812828064


In [55]:
liste_parametres[::-1]

[[900.0,
  (1.6724560630221093,
   0.19477712536582126,
   182.45793267662845,
   136.26223085402196)],
 [800.0,
  (1.8768200063653393,
   0.7706318663880403,
   195.02092555415373,
   125.36861750321283)],
 [700.0,
  (1.9291521027448757,
   -0.5332216028649734,
   -103.11678801091838,
   83.1108032644439)],
 [600.0,
  (1.9602774047188514,
   0.4613297955459015,
   172.744203691743,
   206.90232217669393)],
 [500.0,
  (1.8215809823534128,
   0.2424651944845546,
   -204.081589594055,
   159.39994060800134)],
 [400.0,
  (1.9453015503443534,
   0.43027734184602084,
   -273.60667426160126,
   235.6671495277786)],
 [300.0,
  (1.78687521858346,
   -0.5181309447042584,
   79.42282830523158,
   47.04290993186545)],
 [200.0,
  (1.6112482352805162,
   0.2325530002036118,
   -200.78363678048498,
   237.79346347191301)],
 [100.0,
  (1.1585202444078384,
   0.030629489376269525,
   -146.282394887254,
   212.51282296688686)],
 [99.0,
  (1.8296626455032825,
   0.30878248919136864,
   296.3248392563184

In [ ]:
#not working very well, new distance needed